# Extract food names from burpples reviews
    process in parallel and combine
    save the objets with pickle

    input :
        burpple clean review     burpple__review__clean__20170818.p
    saved: 
        review_with_synsets__20170913.p


## 1. Version Infomation

In [6]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue


## 2. Retrive data

In [7]:
# all possible date when data is saved
date_list = list(set([f.split("__")[-1].replace(".p", "") 
     for f in os.listdir(os.getcwd()) if (f.endswith(".p") and ("__review__clean__") in f)]))
print ("all dates: %s" % ", ".join(date_list))
recent_date = sorted(date_list)[-1]
print ("most recent: %s" % recent_date)

all dates: 20171113, 20170818, 20180205, 20171114
most recent: 20180205


In [8]:
# retrive for testing
def retrive_file(recent_date):
    import pickle
    file_name = "burpple" + "__review__clean__" + recent_date + ".p"
    with open(file_name, 'rb') as pfile:
        df_retrived = pickle.load(pfile)
    print ("retrived: %s" % file_name)
    return df_retrived

In [9]:
%%time
#file_name = "burpple_review__clean__20170818.p"    
df = retrive_file(recent_date)

retrived: burpple__review__clean__20180205.p
CPU times: user 17 s, sys: 268 ms, total: 17.3 s
Wall time: 17.3 s


In [10]:
print df.shape

(323104, 10)


## 3. Review text as unique identifier, get loc list

In [11]:
d = df.groupby('review_text')['loc'].apply(list).to_frame().reset_index()
print d.shape[0]
d.head(2)

286612


,review_text,loc
0,,"[burppleinitial/review/--wu4Z4_, burppleinitia..."
1,!,"[burppleinitial/review/-29WyDL8, burppleinitia..."


## 4. Clean review text

In [13]:
import re
import pandas as pd
pd.options.mode.chained_assignment = None # default is warn

def clean_name_v1(s): 
    try:
        import re
        s = s.replace("\t"," ").replace("\n"," ") 
        s = re.sub(' +',' ', s.strip()) # multiple spaces
        return s
    except:
        return ""


def clean_name_v2(s): 
    import re
    s = clean_name_v1(s)
    s = s.replace("w/o", " no ").replace("W/o", " no ").replace("W/O", " no ").replace("w/O", " no ")
    s = s.replace("w/", " ").replace("W/", " ")
    s = " ".join(re.sub( r"([A-Z])", r" \1", s).split()) #capital letter
    # numeric + character
    s = ' '.join([w for w in s.split() if len(re.findall('[a-zA-Z]+|\\d+', w))==1]) 
    s = re.sub('[^a-zA-Z\n]', ' ', s) # other character 
    s = ' '.join( [w for w in s.split() if len(w)>1] ) #single character
    s = re.sub(' +',' ', s.strip()) # multiple spaces
    return s

def clean_name(s):
    s = clean_name_v2(s).lower()
    return s

In [14]:
%%time
d["clean_text"] = d["review_text"].apply(clean_name)

CPU times: user 38.6 s, sys: 0 ns, total: 38.6 s
Wall time: 38.6 s


In [15]:
""" clean_text as unique identifier , merge loc list"""
d1 = d.groupby("clean_text")['loc'].apply(sum).to_frame().reset_index()
d1 = d1[d1["clean_text"]!=""]
print d1.shape[0]
d1.head(2)

276832


,clean_text,loc
1,aa ahsighee bowl made with frozen aa berry pur...,[burppleinitial/review/huuB1mUX]
2,aa berry bowl must try every aa bowl see on th...,[burppleinitial/review/z_Y7aErb]


In [ ]:
""" save into multiple files """

In [16]:
df_lst = []
import numpy as np
for g, df in d1.groupby(np.arange(len(d1)) // 10000):
    df_lst.append(df)
print len(df_lst)

28


In [17]:
def save_part(i, df_part):
    import pickle
    import datetime   
    today = datetime.date.today()
    today = today.strftime('%Y%m%d') 
    file_name = "__".join(["review", today, str(i)]) + ".p"  
    with open(file_name, 'wb') as pfile:
        pickle.dump(df_part, pfile)
    print "saved: %s"%file_name

In [18]:
for i in range(len(df_lst)):
    df_part = df_lst[i]
    save_part(i, df_part)

saved: review__20180205__0.p
saved: review__20180205__1.p
saved: review__20180205__2.p
saved: review__20180205__3.p
saved: review__20180205__4.p
saved: review__20180205__5.p
saved: review__20180205__6.p
saved: review__20180205__7.p
saved: review__20180205__8.p
saved: review__20180205__9.p
saved: review__20180205__10.p
saved: review__20180205__11.p
saved: review__20180205__12.p
saved: review__20180205__13.p
saved: review__20180205__14.p
saved: review__20180205__15.p
saved: review__20180205__16.p
saved: review__20180205__17.p
saved: review__20180205__18.p
saved: review__20180205__19.p
saved: review__20180205__20.p
saved: review__20180205__21.p
saved: review__20180205__22.p
saved: review__20180205__23.p
saved: review__20180205__24.p
saved: review__20180205__25.p
saved: review__20180205__26.p
saved: review__20180205__27.p


## 5. clean review text: replace by synsets

In [36]:
def retrive_part(i):
    import os
    import pickle
    prefix = "review"    
    date_list = list(set([f.split("__")[-2].replace(".p", "") 
         for f in os.listdir(os.getcwd()) if (f.endswith(".p") and (prefix) in f)]))
    recent_date = sorted([s for s in date_list if len(s)==8])[-1]
    file_name = "__".join([prefix, recent_date, str(i)]) + ".p"
    with open(file_name, 'rb') as pfile:
        retrived = pickle.load(pfile)       
    print ("retrived: %s" % file_name)
    print "number of reviews: %d"%len(retrived)
    return retrived

def save_part_with_food(i, samp):
    import pickle
    import datetime   
    today = datetime.date.today()
    today = today.strftime('%Y%m%d') 
    file_name = "__".join(["review_with_food", today, str(i)]) + ".p"  
    with open(file_name, 'wb') as pfile:
        pickle.dump(samp, pfile)
    print "saved: %s"%file_name
    
def retrive_file(file_name):
    import pickle
    with open(file_name, 'rb') as pfile:
        df_retrived = pickle.load(pfile)
    print ("retrived: %s" % file_name)
    return df_retrived

In [37]:
i = 0
samp = retrive_part(i)

retrived: review__20180205__0.p
number of reviews: 10000


In [41]:
samp.head(2)

,clean_text,loc
1,aa ahsighee bowl made with frozen aa berry pur...,[burppleinitial/review/huuB1mUX]
2,aa berry bowl must try every aa bowl see on th...,[burppleinitial/review/z_Y7aErb]


In [44]:
import spacy
nlp = spacy.load('en_core_web_sm')

def lemma_name(s):
    return " ".join([token.lemma_ for token in nlp(unicode(s, "utf-8"))])

""" to replace syn tokens """
def retrive_syn_tokens():
    import os
    import pickle
    prefix = "syn_token"  
    token_path = "/home/yueliu/Desktop/workspace_yue/Documentation_201712"
    date_list = list(set([f.split("__")[-1].replace(".p", "")
                          for f in os.listdir(token_path) if (f.endswith(".p")
                                                             and prefix in f)]))
    recent_date = sorted(date_list)[-1]

    file_name = os.path.join(token_path,"__".join([prefix, recent_date]) + ".p")
    with open(file_name, 'rb') as pfile:
        retrived = pickle.load(pfile)       
    print ("retrived: %s" % file_name)
    print "number of syn tokens: %d"%len(retrived)
    return retrived

def rev_dict(x):
    """ return the first element in lst as value """
    return dict((x[i],x[0]) for i in range(1, len(x)))
def merge_dicts(d_lst):
    """ shallow copy and merge into new dict"""
    result = {}
    for dictionary in d_lst:
        result.update(dictionary)
    return result   

def replace_token(s):
    """ partial string matching """
    s = " " + s + " "
    s = s.replace(" ", "  ")
    for key in keys:       
        s = s.replace(" " + key + " ", " " + rev_lookup[key] + " ")
    import re
    s = re.sub(' +',' ', s.strip()) # multiple spaces    
    return s 

syn_tokens = retrive_syn_tokens()
rev_lookup = merge_dicts([rev_dict(x) for x in syn_tokens])  
keys = sorted(list(set(rev_lookup.keys())),key=lambda x: -len(x))

retrived: /home/yueliu/Desktop/workspace_yue/Documentation_201712/syn_token__20180207.p
number of syn tokens: 1177


In [46]:
""" get food names from standard food names"""
token_path = "/home/yueliu/Desktop/workspace_yue/Documentation_201712"
file_name = os.path.join(token_path,"all_food_vendor__standard__20180207.p")
food_names = retrive_file(file_name)
food_names = sorted(food_names["standard_name"].unique(), key=lambda (x): -len(x))

retrived: /home/yueliu/Desktop/workspace_yue/Documentation_201712/all_food_vendor__standard__20180207.p


In [50]:
print "number of food items: %d"%len(food_names1)

number of food items: 92298


In [56]:
samp.head(2)

,clean_text,loc
1,aa ahsighee bowl made with frozen aa berry pur...,[burppleinitial/review/huuB1mUX]
2,aa berry bowl must try every aa bowl see on th...,[burppleinitial/review/z_Y7aErb]


In [58]:
%%time
samp["lemma_text"] = samp["clean_text"].apply(lemma_name)

CPU times: user 6min 9s, sys: 19min 56s, total: 26min 6s
Wall time: 3min 18s


In [59]:
%%time
samp["standard_text"] = samp["lemma_text"].apply(replace_token)

CPU times: user 17.3 s, sys: 280 ms, total: 17.6 s
Wall time: 17.3 s


## 6. check if food names are in review higher_text

In [49]:
# search in a review, list down the possible food names (with attributes) it contain
# longest matching food items
def search_clean_name(searchFor):
    searchFor= " " + searchFor+ " "
    vs = []
    for v in food_names:
        if " " + v+ " " in searchFor:
            vs.append(v)
    v_unique = longest_unique_entity(vs)  
    return v_unique

# return "chicken rice" and "fish soup" form ["chicken rice", "chicken", "fish soup", "soup"]
def longest_unique_entity(lst):
    lst = list(set(lst))
    lst1 = deepcopy(lst)
    for i in lst:
        for j in lst:
            if (len(i)<len(j)) and (i in j):
                try:
                    lst1.remove(i)
                except:
                    pass
    return tuple(lst1)

In [60]:
%%time
samp["food_items"] = samp["standard_text"].apply(search_clean_name)

CPU times: user 9min 20s, sys: 6.44 s, total: 9min 27s
Wall time: 9min 20s


In [61]:
save_part_with_food(100+i, samp)

saved: review_with_food__20180207__100.p


In [62]:
samp.head(2)

,clean_text,loc,lemma_text,standard_text,food_items
1,aa ahsighee bowl made with frozen aa berry pur...,[burppleinitial/review/huuB1mUX],aa ahsighee bowl make with frozen aa berry pur...,aa ahsighee bowl make with frozen aa berry pur...,"(superfood, and, omega, pure)"
2,aa berry bowl must try every aa bowl see on th...,[burppleinitial/review/z_Y7aErb],aa berry bowl must try every aa bowl see on th...,aa berry bowl must try every aa bowl see on th...,"(smoothie bowl,)"


### process all parts

In [76]:
def retrive_part(i):
    import os
    import pickle
    prefix = "review"    
#     date_list = list(set([f.split("__")[-2].replace(".p", "") 
#          for f in os.listdir(os.getcwd()) if (f.endswith(".p") and (prefix) in f)]))
#     recent_date = sorted([s for s in date_list if len(s)==8])[-1]
    recent_date = "20180205"
    file_name = "__".join([prefix, recent_date, str(i)]) + ".p"
    with open(file_name, 'rb') as pfile:
        retrived = pickle.load(pfile)       
    print ("retrived: %s" % file_name)
    print "number of reviews: %d"%len(retrived)
    return retrived

In [ ]:
%%time
## started: 7/2/2018 1100, estimated completation time: 1800
for i in range(1,28):
    samp = retrive_part(i)
    samp["lemma_text"] = samp["clean_text"].apply(lemma_name)
    samp["standard_text"] = samp["lemma_text"].apply(replace_token)
    samp["food_items"] = samp["standard_text"].apply(search_clean_name)
    save_part_with_food(100+i, samp)

retrived: review__20180205__1.p
number of reviews: 10000


## 7. review_with_synsets

In [49]:
"""retrive parts & merge into singel df"""
# prefix = "review_with_food__20180205__"
# # all possible date when data is saved
# file_lst = [f for f in os.listdir(os.getcwd()) if (f.endswith(".p") and (prefix) in f)]
# print len(file_lst)

# # retrive for testing
# def retrive_file(file_name):
#     import pickle
#     with open(file_name, 'rb') as pfile:
#         df_retrived = pickle.load(pfile)
#     print ("retrived: %s" % file_name)
#     return df_retrived
# df_lst = [retrive_file(file_name) for file_name in file_lst]
# df1 = pd.concat(df_lst)

28


In [65]:
from copy import deepcopy
df1 = deepcopy(samp)

In [66]:
%%time 
# about 5.5hrs for all reviews
df1 = df1[df1["food_items"].map(len)>0]
col = "food_items"
# split into multiple rows
s = df1[col].apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = col 
del df1[col]
df1 = df1.join(s)

CPU times: user 2.01 s, sys: 0 ns, total: 2.01 s
Wall time: 2.02 s


In [67]:
print df1.shape
print df1.columns.tolist()
df1.head(2)

(37653, 5)
['clean_text', 'loc', 'lemma_text', 'standard_text', 'food_items']


,clean_text,loc,lemma_text,standard_text,food_items
1,aa ahsighee bowl made with frozen aa berry pur...,[burppleinitial/review/huuB1mUX],aa ahsighee bowl make with frozen aa berry pur...,aa ahsighee bowl make with frozen aa berry pur...,superfood
1,aa ahsighee bowl made with frozen aa berry pur...,[burppleinitial/review/huuB1mUX],aa ahsighee bowl make with frozen aa berry pur...,aa ahsighee bowl make with frozen aa berry pur...,and


In [72]:
def save_file(df, desc):
    import pickle
    import datetime   
    today = datetime.date.today()
    today = today.strftime('%Y%m%d') 
    file_name = "__".join([desc, today]) + ".p"  
    with open(file_name, 'wb') as pfile:
        pickle.dump(df, pfile)
    print "saved: %s"%file_name
save_file(df1, "review_with_synsets")

saved: review_with_synsets__20180207.p


# Evaluation 

In [70]:
from collections import Counter
counts = Counter(df1["food_items"].tolist())
counts.most_common(20)

[(u'and', 5122),
 (u'for', 2929),
 (u'good', 1709),
 (u'food', 842),
 (u'sweet', 631),
 (u'out', 611),
 (u'flavour', 546),
 (u'sauce', 523),
 (u'singapore', 388),
 (u'back', 386),
 (u'favourite', 363),
 (u'ice cream', 353),
 (u'week', 309),
 (u'crispy', 262),
 (u'tender', 225),
 (u'egg', 216),
 (u'meat', 212),
 (u'mixed', 201),
 (u'any', 195),
 (u'choice', 176)]

In [71]:
des_key_remove = ["and", "for", "good", "food", "out", 'flavour', 'sauce','singapore', 
                 'back', "any", "choice", "week"]
df2 = df1[~df1["food_items"].isin(des_key_remove)]
print "number of synsets removed: %d" % len(des_key_remove)
print "number of clean_name in reviews: %d" % df2.shape[0] 
#print "number of reviews with clean_name(s): %d" % df2["loc"].nunique()
print "number of unique review text identified: %d" % df2["clean_text"].nunique()
print "number of unique synset identified: %d" % df2["food_items"].nunique()

number of synsets removed: 12
number of clean_name in reviews: 23917
number of unique review text identified: 7504
number of unique synset identified: 4332
